In [19]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
import brunoflow as bf
from brunoflow.ad.utils import check_node_equals_tensor, check_node_allclose_tensor
from jax import numpy as jnp
import transformers
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    BertForMaskedLM, 
    BertTokenizer, 
    BertTokenizerFast, 
    BertEmbeddings,
    BfBertEmbeddings,
    BfBertEncoder,
    BertConfig,
    BfBertSelfAttention,
)
from collections import Counter

torch.manual_seed(0)

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [2]:
# Establish data
model_id = "google/bert_uncased_L-2_H-128_A-2"
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-tiny.json")
tokenizer = BertTokenizerFast.from_pretrained(model_id)
text = ["hello I want to eat some [MASK] meat today. It's thanksgiving [MASK] all!", "yo yo what's up"]

# tokenize text and pass into model
tokens = tokenizer(text, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"]
jax_input_ids = bf.Node(jnp.array(input_ids.numpy(), dtype=int), name="inputs")
print(input_ids)

2023-01-10 13:11:30.187775: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


tensor([[  101,  7592,  1045,  2215,  2000,  4521,  2070,   103,  6240,  2651,
          1012,  2009,  1005,  1055, 15060,   103,  2035,   999,   102],
        [  101, 10930, 10930,  2054,  1005,  1055,  2039,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [3]:
# Create BfBertEmbeddings and BertEmbeddings
bf_embs = BfBertEmbeddings(config)
# Visualize output of forward pass of BfBertEmbeddings
bf_embs.train(False)
out_bf = bf_embs(input_ids=jax_input_ids)
# out_bf.visualize(collapse_to_modules=True)
# print(bf_embs)
bf_embs.train(True)
out_bf.backprop(values_to_compute=("max_grad",))

/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:178: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/jax/_src/ops/scatter.py:87: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "
/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/jax/_src/ops/scatter.py:87: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=int32. In future JAX releases this will result in an error.
  warnings.warn("scatter inputs have incompatible types: cannot safely cast "


In [4]:
out_bf.visualize()

In [5]:
input_ids[0]

tensor([  101,  7592,  1045,  2215,  2000,  4521,  2070,   103,  6240,  2651,
         1012,  2009,  1005,  1055, 15060,   103,  2035,   999,   102])

In [6]:
import numpy as np
bf_embs.word_embeddings.weight.get_max_grad_parent().shape

(30522, 128)

In [7]:
bf_embs.word_embeddings.weight.get_max_grad_parent()[101].shape

(128,)

In [16]:
bf_embs.word_embeddings.weight.get_parents()[0].get_parents()[0].get_parents()[0].get_parents()

[node(name: sum, val: [[[ -8.037544 ]
   [ -9.25424  ]
   [-30.422623 ]
   [-23.394344 ]
   [ 18.407639 ]
   [-15.804301 ]
   [ -5.4714947]
   [-14.878903 ]
   [ -2.0510278]
   [-15.913745 ]
   [ -1.8737106]
   [  9.090008 ]
   [-36.69632  ]
   [-21.18118  ]
   [-43.435745 ]
   [ -1.92349  ]
   [-15.039597 ]
   [  1.0616264]
   [ -9.414511 ]]
 
  [[ -8.037544 ]
   [-21.553772 ]
   [-31.694557 ]
   [-41.051914 ]
   [  1.2789683]
   [-17.676403 ]
   [-41.590836 ]
   [ -3.2156487]
   [ -5.3564873]
   [-22.55719  ]
   [ -0.2202673]
   [  8.048849 ]
   [-28.457026 ]
   [-16.717178 ]
   [-20.629799 ]
   [  9.942659 ]
   [-10.314689 ]
   [ -9.4309025]
   [ -9.211617 ]]], grad: [[[0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]]
 
  [[0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]
   [0.]]]),
 node(name:

In [22]:
Counter(bf_embs.word_embeddings.weight.get_parents()[0].get_parents()[0].get_parents()[0].get_max_grad_parent()[0, 0])

Counter({node(name: -, val: [[[-2.055826    1.9602232  -2.4997613  ... -0.66506654  1.7868567
            -2.0468354 ]
           [ 0.91205066  1.3642676  -0.27127576 ... -3.654943    2.5712256
             2.820441  ]
           [-2.9715686  -1.9735163  -0.75190353 ... -0.6736827  -2.6053133
             2.7153106 ]
           ...
           [-1.8908731  -0.90543425 -2.153562   ...  0.0360316  -0.4378655
            -1.4195855 ]
           [-2.36614     2.1955698  -4.0641236  ... -1.8886303  -0.25730273
            -0.5975659 ]
           [ 0.9218515   1.6637299  -1.3754494  ... -1.7345631   1.5255744
             0.5460858 ]]
         
          [[-2.055826    1.9602232  -2.4997613  ... -0.66506654  1.7868567
            -2.0468354 ]
           [-1.8819065   4.485992   -0.54169416 ... -2.1072445   1.3272808
             1.9330618 ]
           [-2.0620303   1.2712848  -1.3479426  ...  0.69896513 -0.69501644
             1.7687442 ]
           ...
           [-0.6520039   0.3093468  -1

In [ ]:
Counter(bf_embs.word_embeddings.weight.get_max_grad_parent()[101][0].get_max_grad_parent()[0][0][0].get_max_grad_parent()[0][0][0].get_max_grad_parent()[0][0][0].get_max_grad_parent()[0][0][0].get_max_grad_parent()[0][0])

Counter({node(name: *, val: [[[-1.3068492   1.2460763  -1.5890504  ... -0.42277008  1.1358706
            -1.3011341 ]
           [ 0.5956796   0.891032   -0.17717592 ... -2.3871205   1.6793218
             1.8420895 ]
           [-1.698645   -1.1281259  -0.42981243 ... -0.38509887 -1.4892815
             1.5521595 ]
           ...
           [-1.1109186  -0.5319573  -1.2652527  ...  0.02116915 -0.25725308
            -0.8340295 ]
           [-1.3572129   1.2593743  -2.331173   ... -1.0833144  -0.14758831
            -0.34276256]
           [ 0.60616684  1.0939916  -0.9044318  ... -1.1405684   1.003147
             0.3590807 ]]
         
          [[-1.3068492   1.2460763  -1.5890504  ... -0.42277008  1.1358706
            -1.3011341 ]
           [-1.0704961   2.5517936  -0.30813512 ... -1.1986765   0.755005
             1.0995951 ]
           [-1.2653587   0.78012013 -0.82716095 ...  0.42891786 -0.42649475
             1.0853846 ]
           ...
           [-0.47345397  0.22463281 -1.

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].name

'get_embedding'

In [ ]:
len(bf_embs.word_embeddings.weight.parents), bf_embs.word_embeddings.weight.parents, list(bf_embs.word_embeddings.weight.parents)[0].shape

(1,
 {node(name: get_embedding, val: [[[-0.14575319 -0.19785921 -0.15279959 ...  1.1640646   0.2520253
     -1.8681448 ]
    [ 1.953604   -0.45420492 -0.45989478 ... -0.26960897  1.3368154
      0.84715754]
    [-1.8360448  -0.6634346  -0.22042722 ... -0.18071136 -1.9035796
      0.8201915 ]
    ...
    [-1.2757827  -1.2516944  -0.4720613  ...  0.13241069 -0.6340039
     -0.96398574]
    [-0.6756279   0.53763574 -1.1274053  ...  0.10094006 -0.6436071
      0.2731806 ]
    [ 0.98025113  1.2585802   0.35818103 ...  0.21049303  0.47360757
      0.30244443]]
  
   [[-0.14575319 -0.19785921 -0.15279959 ...  1.1640646   0.2520253
     -1.8681448 ]
    [-0.93644345  2.5714295  -0.82640326 ...  1.1819994  -0.00321964
     -0.13631164]
    [-0.93644345  2.5714295  -0.82640326 ...  1.1819994  -0.00321964
     -0.13631164]
    ...
    [ 0.          0.          0.         ...  0.          0.
      0.        ]
    [ 0.          0.          0.         ...  0.          0.
      0.        ]
    [ 0.  

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1].shape

'get_embedding'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].name

'+'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1].shape

(2, 19, 128)

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].name

'+'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].name

'-'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].name

'/'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].name

'*'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].name

'+'

In [ ]:
bf_embs.word_embeddings.weight.max_grad_of_output_wrt_node[1][101][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].max_grad_of_output_wrt_node[1][0][0][0].parents

set()